#HW 1

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

2) Изменим регистр твитов на нижний с помощью .lower().

3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

4) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

9) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

11) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

14) Сохраним результат предобработки в pickle-файл.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/test_tweets.csv')
train_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [4]:
test_df = pd.read_csv('/content/drive/MyDrive/train_tweets.csv')
test_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,@user #white #supremacists want everyone to s...,NaN
2,31965,safe ways to heal your #acne!! #altwaystohe...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",NaN


In [6]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   tweet   49159 non-null  object 
 2   label   31962 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


In [7]:
"""create class for preprocessing"""

class TextPreprocessor:
    
    stop_words = nltk.corpus.stopwords.words('english')
    stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    def __init__(self, df: pd.DataFrame, column: str):
        
        self.df = df
        self.column = column
        
    def resub_patt(self, pattern: str, repl: str = '') -> np.array:
        
        pattern = re.compile(pattern=pattern) # init series pattern
        
        vector = np.vectorize(pattern.sub) # init resub series vector
        result = vector(repl=repl, string=self.df[self.column]) # resub series
        
        self.df[self.column] = result
        
        return result
    
    def reg_series(self, register: str = 'lower') -> np.array:
        
        vector = np.vectorize(str.lower if register == 'lower' else str.upper)
        result = vector(self.df[self.column])
        
        self.df[self.column] = result
        
        return result
    
    def repl_abb(self, repl_dict: dict) -> list:
    
        result = [text.split() for text in self.df[self.column]]

        for id_wl, words in enumerate(result): # перебор списков со словами
            for id_w, word in enumerate(words): # перебор слов
                result[id_wl][id_w] = repl_dict[word] if word in repl_dict.keys() else word

            result[id_wl] = ' '.join(result[id_wl])
            
        self.df[self.column] = result

        return result
    
    def clean_by_len(self, len_word: int = 1) -> list:
        
        result = [text.split() for text in self.df[self.column]]

        for id_wl, words in enumerate(result): # перебор списков со словами
            result[id_wl] = ' '.join([word for word in words if len(word) > len_word])
        
        self.df[self.column] = result
        
        return result
    
    def tokenize(self, method: str = 'split', new_column: str = None) -> list:
        
        if method == 'split':
            result = [text.split() for text in self.df[self.column]]
        elif method == 'tokenize':
            result = [nltk.tokenize.word_tokenize(text) for text in self.df[self.column]]
        
        if new_column:
            self.df[new_column] = result
        
        return result
    
    def clean_stop_words(self, target: str = None , new_column: str = None):
        
        result = [words for words in self.df[target]]

        for id_wl, words in enumerate(result): # перебор списков со словами
            result[id_wl] = [word for word in words if word not in self.stop_words]

        if new_column:
            self.df[new_column] = result
        
        return result
    
    def stemming(self, target: str = None, new_column: str = None):
        
        result = [words for words in self.df[target]]
        
        for id_wl, words in enumerate(result): # перебор списков со словами
            result[id_wl] = [self.stemmer.stem(word) for word in words]

        if new_column:
            self.df[new_column] = result
        
        return result
    
    def lemmating(self, target: str = None, new_column: str = None):
        
        result = [words for words in self.df[target]]
        
        for id_wl, words in enumerate(result): # перебор списков со словами
            result[id_wl] = [self.lemmatizer.lemmatize(word, nltk.corpus.wordnet.VERB) for word in words]

        if new_column:
            self.df[new_column] = result
        
        return result

In [8]:
"""init text preprocessor for column "tweet" in full_df"""

text_prepr = TextPreprocessor(df=combine_df, column='tweet')

In [9]:
text_prepr.resub_patt(pattern="@[\w]*")
text_prepr.df

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,#white #supremacists want everyone to see th...,NaN
2,31965,safe ways to heal your #acne!! #altwaystohe...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",NaN
...,...,...,...
49154,31958,ate isz that youuu?ððððððð...,0.0
49155,31959,to see nina turner on the airwaves trying to...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,"#sikh #temple vandalised in in #calgary, #wso...",1.0


3. Изменим регистр твитов на нижний с помощью .lower()

In [10]:
text_prepr.reg_series()
text_prepr.df

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,#white #supremacists want everyone to see th...,NaN
2,31965,safe ways to heal your #acne!! #altwaystohe...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",NaN
...,...,...,...
49154,31958,ate isz that youuu?ððððððð...,0.0
49155,31959,to see nina turner on the airwaves trying to...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,"#sikh #temple vandalised in in #calgary, #wso...",1.0


4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [11]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [12]:
text_prepr.repl_abb(repl_dict=apostrophe_dict)
text_prepr.df

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,#white #supremacists want everyone to see the ...,NaN
2,31965,safe ways to heal your #acne!! #altwaystoheal ...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew e...",NaN
...,...,...,...
49154,31958,ate isz that youuu?ððððððð...,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,"#sikh #temple vandalised in in #calgary, #wso ...",1.0


5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [13]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [14]:
text_prepr.repl_abb(repl_dict=short_word_dict)
text_prepr.df

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,#white #supremacists want everyone to see the ...,NaN
2,31965,safe ways to heal your #acne!! #altwaystoheal ...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew e...",NaN
...,...,...,...
49154,31958,ate isz that youuu?ððððððð...,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,"#sikh #temple vandalised in in #calgary, #wso ...",1.0


6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [15]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [16]:
text_prepr.repl_abb(repl_dict=emoticon_dict)
text_prepr.df

,id,tweet,label
0,31963,#studiolife #aislife #requires #passion #dedic...,NaN
1,31964,#white #supremacists want everyone to see the ...,NaN
2,31965,safe ways to heal your #acne!! #altwaystoheal ...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,"3rd #bihday to my amazing, hilarious #nephew e...",NaN
...,...,...,...
49154,31958,ate isz that youuu?ððððððð...,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,"#sikh #temple vandalised in in #calgary, #wso ...",1.0


7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [17]:
text_prepr.resub_patt(pattern=r'[^\w\s]')
text_prepr.df

,id,tweet,label
0,31963,studiolife aislife requires passion dedication...,NaN
1,31964,white supremacists want everyone to see the ne...,NaN
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,3rd bihday to my amazing hilarious nephew eli ...,NaN
...,...,...,...
49154,31958,ate isz that youuuðððððððððâï,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [18]:
text_prepr.resub_patt(pattern=r'[^a-zA-Z0-9]', repl=' ')
text_prepr.df

,id,tweet,label
0,31963,studiolife aislife requires passion dedication...,NaN
1,31964,white supremacists want everyone to see the ne...,NaN
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,3rd bihday to my amazing hilarious nephew eli ...,NaN
...,...,...,...
49154,31958,ate isz that youuu,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0


9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [19]:
text_prepr.resub_patt(pattern=r'[^a-zA-Z]', repl=' ')
text_prepr.df

,id,tweet,label
0,31963,studiolife aislife requires passion dedication...,NaN
1,31964,white supremacists want everyone to see the ne...,NaN
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,rd bihday to my amazing hilarious nephew eli ...,NaN
...,...,...,...
49154,31958,ate isz that youuu,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on a monday morning otw...,0.0
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0


10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [20]:
text_prepr.clean_by_len()
text_prepr.df

,id,tweet,label
0,31963,studiolife aislife requires passion dedication...,NaN
1,31964,white supremacists want everyone to see the ne...,NaN
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN
3,31966,is the hp and the cursed child book up for res...,NaN
4,31967,rd bihday to my amazing hilarious nephew eli a...,NaN
...,...,...,...
49154,31958,ate isz that youuu,0.0
49155,31959,to see nina turner on the airwaves trying to w...,0.0
49156,31960,listening to sad songs on monday morning otw t...,0.0
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0


11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [21]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
text_prepr.tokenize(method='tokenize', new_column='tweet_token')
text_prepr.df

,id,tweet,label,tweet_token
0,31963,studiolife aislife requires passion dedication...,NaN,"[studiolife, aislife, requires, passion, dedic..."
1,31964,white supremacists want everyone to see the ne...,NaN,"[white, supremacists, want, everyone, to, see,..."
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN,"[safe, ways, to, heal, your, acne, altwaystohe..."
3,31966,is the hp and the cursed child book up for res...,NaN,"[is, the, hp, and, the, cursed, child, book, u..."
4,31967,rd bihday to my amazing hilarious nephew eli a...,NaN,"[rd, bihday, to, my, amazing, hilarious, nephe..."
...,...,...,...,...
49154,31958,ate isz that youuu,0.0,"[ate, isz, that, youuu]"
49155,31959,to see nina turner on the airwaves trying to w...,0.0,"[to, see, nina, turner, on, the, airwaves, try..."
49156,31960,listening to sad songs on monday morning otw t...,0.0,"[listening, to, sad, songs, on, monday, mornin..."
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0,"[sikh, temple, vandalised, in, in, calgary, ws..."


12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [23]:
text_prepr.clean_stop_words(target='tweet_token', new_column='tweet_token_filtered')
text_prepr.df

,id,tweet,label,tweet_token,tweet_token_filtered
0,31963,studiolife aislife requires passion dedication...,NaN,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic..."
1,31964,white supremacists want everyone to see the ne...,NaN,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new..."
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health..."
3,31966,is the hp and the cursed child book up for res...,NaN,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread..."
4,31967,rd bihday to my amazing hilarious nephew eli a...,NaN,"[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ..."
...,...,...,...,...,...
49154,31958,ate isz that youuu,0.0,"[ate, isz, that, youuu]","[ate, isz, youuu]"
49155,31959,to see nina turner on the airwaves trying to w...,0.0,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma..."
49156,31960,listening to sad songs on monday morning otw t...,0.0,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ..."
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0,"[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde..."


13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [24]:
text_prepr.stemming(target='tweet_token', new_column='tweet_stemmed')
text_prepr.df

,id,tweet,label,tweet_token,tweet_token_filtered,tweet_stemmed
0,31963,studiolife aislife requires passion dedication...,NaN,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic...","[studiolif, aislif, requir, passion, dedic, wi..."
1,31964,white supremacists want everyone to see the ne...,NaN,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new...","[white, supremacist, want, everyon, to, see, t..."
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health...","[safe, way, to, heal, your, acn, altwaystoh, h..."
3,31966,is the hp and the cursed child book up for res...,NaN,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread...","[is, the, hp, and, the, curs, child, book, up,..."
4,31967,rd bihday to my amazing hilarious nephew eli a...,NaN,"[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ...","[rd, bihday, to, my, amaz, hilari, nephew, eli..."
...,...,...,...,...,...,...
49154,31958,ate isz that youuu,0.0,"[ate, isz, that, youuu]","[ate, isz, youuu]","[ate, isz, that, youuu]"
49155,31959,to see nina turner on the airwaves trying to w...,0.0,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma...","[to, see, nina, turner, on, the, airwav, tri, ..."
49156,31960,listening to sad songs on monday morning otw t...,0.0,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ...","[listen, to, sad, song, on, monday, morn, otw,..."
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0,"[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde...","[sikh, templ, vandalis, in, in, calgari, wso, ..."


14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [25]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [26]:
text_prepr.lemmating(target='tweet_token', new_column='tweet_lemmatized')
text_prepr.df

,id,tweet,label,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,31963,studiolife aislife requires passion dedication...,NaN,"[studiolife, aislife, requires, passion, dedic...","[studiolife, aislife, requires, passion, dedic...","[studiolif, aislif, requir, passion, dedic, wi...","[studiolife, aislife, require, passion, dedica..."
1,31964,white supremacists want everyone to see the ne...,NaN,"[white, supremacists, want, everyone, to, see,...","[white, supremacists, want, everyone, see, new...","[white, supremacist, want, everyon, to, see, t...","[white, supremacists, want, everyone, to, see,..."
2,31965,safe ways to heal your acne altwaystoheal heal...,NaN,"[safe, ways, to, heal, your, acne, altwaystohe...","[safe, ways, heal, acne, altwaystoheal, health...","[safe, way, to, heal, your, acn, altwaystoh, h...","[safe, ways, to, heal, your, acne, altwaystohe..."
3,31966,is the hp and the cursed child book up for res...,NaN,"[is, the, hp, and, the, cursed, child, book, u...","[hp, cursed, child, book, reservations, alread...","[is, the, hp, and, the, curs, child, book, up,...","[be, the, hp, and, the, curse, child, book, up..."
4,31967,rd bihday to my amazing hilarious nephew eli a...,NaN,"[rd, bihday, to, my, amazing, hilarious, nephe...","[rd, bihday, amazing, hilarious, nephew, eli, ...","[rd, bihday, to, my, amaz, hilari, nephew, eli...","[rd, bihday, to, my, amaze, hilarious, nephew,..."
...,...,...,...,...,...,...,...
49154,31958,ate isz that youuu,0.0,"[ate, isz, that, youuu]","[ate, isz, youuu]","[ate, isz, that, youuu]","[eat, isz, that, youuu]"
49155,31959,to see nina turner on the airwaves trying to w...,0.0,"[to, see, nina, turner, on, the, airwaves, try...","[see, nina, turner, airwaves, trying, wrap, ma...","[to, see, nina, turner, on, the, airwav, tri, ...","[to, see, nina, turner, on, the, airwaves, try..."
49156,31960,listening to sad songs on monday morning otw t...,0.0,"[listening, to, sad, songs, on, monday, mornin...","[listening, sad, songs, monday, morning, otw, ...","[listen, to, sad, song, on, monday, morn, otw,...","[listen, to, sad, songs, on, monday, morning, ..."
49157,31961,sikh temple vandalised in in calgary wso conde...,1.0,"[sikh, temple, vandalised, in, in, calgary, ws...","[sikh, temple, vandalised, calgary, wso, conde...","[sikh, templ, vandalis, in, in, calgari, wso, ...","[sikh, temple, vandalise, in, in, calgary, wso..."


15. Сохраним результат предобработки в pickle-файл.

In [28]:
text_prepr.df.to_pickle(path='/content/Text/full_words_prep.pkl')